In [ ]:
!pip install tensorflow_text

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd
import csv

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
def tf_lower_and_split_punct(text):
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)
  return text

In [70]:
def create_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="sentences")
  preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name="preprosesing")(text_input)
  encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=False, name="embedding")(preprocessor)
  # pooled_output = outputs["pooled_output"]# [batch_size, 768].
  # sequence_output = outputs["sequence_output"]# [batch_size, seq_length, 768].
  x1 = tf.keras.layers.Dense(1024, activation="relu")(encoder["pooled_output"])
  x2 = tf.keras.layers.Dropout(0.1)(x1)
  x3 = tf.keras.layers.Dense(1, activation ="sigmoid", name="output_layer")(x2)

  model = tf.keras.Model(inputs=[text_input], outputs=[x3], name="Binary_Classification_Model")

  model.compile(
      optimizer="adam",
      loss = tf.keras.losses.BinaryCrossentropy(),
      metrics=["accuracy"]
  )

  model.summary()
  return model

with open("/content/drive/MyDrive/IOH/spam_or_not/data/emails.csv") as f:
  csvreader = csv.reader(f, delimiter=",")
  next(csvreader)
  masukan = []
  keluaran = []
  for a, i in enumerate(csvreader):
      inp1 = tf_lower_and_split_punct(i[0]).numpy().decode()
      inp2 = tf_lower_and_split_punct(i[2]).numpy().decode()
      tar = int(i[1])
      masukan.append(inp1)
      masukan.append(inp2)
      keluaran.append(int(tar))
      keluaran.append(int(tar))

BUFFER_SIZE = len(masukan)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((masukan, keluaran)).shuffle(BUFFER_SIZE)
train = dataset.take(int(0.7*len(masukan)))
remaining = dataset.skip(int(0.7*len(masukan)))
validation = remaining.take(int(0.15*len(masukan)))
test = remaining.skip(int(0.15*len(masukan)))

train = train.batch(BATCH_SIZE)
validation = validation.batch(BATCH_SIZE)

In [53]:
for example_input_batch, example_target_batch in train.take(1):
  print(example_input_batch[:5])
  print()
  print(example_target_batch[:5])
  break

tf.Tensor(
[b'pesan untuk peserta kursus lacima ,  julie ,  terima kasih .  juga ,  sebagai tindak lanjut apakah anda menerima cek dari paul quilkey ?  vince julie pada     pm harap tanggapi julie cc buku untuk peserta kursus lacima hanya ingin memberi tahu anda bahwa turunan energi  ?  buku harga dan manajemen risiko ,  oleh clewlow dan strickland dan disponsori oleh enron corp .   ,  ?  selesai .   ?  kami akan mulai pengiriman  januari ,  yang akan termasuk distribusi ?  salinan gratis anda .   ?  terima kasih atas kesabaran dan dukungan anda .   ?  hormat kami ,  julie ?  grup lacima ?   ?'
 b'meet during cmu visit  ?   aziz  ,   please  ,  contact me before or after the presentation and we can find  a time slot to chat later on friday  .   vince  al  v on           pm  vince  .  j  .  kaminski  enron  .  com  cc   meet during cmu visit  ?   dr  .  kaminski   i am a ph  .  d  .  student here at carnegie mellon who has a long  standing  interest in the energy industry  .   i have re

In [71]:
model = create_model()
ckpt_path = "/content/drive/MyDrive/IOH/spam_or_not/checkpoint/model.{epoch:02d}-{val_loss:.4f}.h5"
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=ckpt_path, 
    save_weights_only=True,
    save_best_only=True,
    verbose=1, 
)

Model: "Binary_Classification_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sentences (InputLayer)         [(None,)]            0           []                               
                                                                                                  
 preprosesing (KerasLayer)      {'input_type_ids':   0           ['sentences[0][0]']              
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                            

In [72]:
model.fit(train, validation_data=validation, epochs=15, callbacks=[ckpt])

Epoch 1/15
126/126 [==============================] - ETA: 0s - loss: 0.5428 - accuracy: 0.7568
Epoch 1: val_loss improved from inf to 0.49062, saving model to /content/drive/MyDrive/IOH/spam_or_not/checkpoint/model.01-0.4906.h5
126/126 [==============================] - 126s 932ms/step - loss: 0.5428 - accuracy: 0.7568 - val_loss: 0.4906 - val_accuracy: 0.7905
Epoch 2/15
126/126 [==============================] - ETA: 0s - loss: 0.3668 - accuracy: 0.8370
Epoch 2: val_loss improved from 0.49062 to 0.34736, saving model to /content/drive/MyDrive/IOH/spam_or_not/checkpoint/model.02-0.3474.h5
126/126 [==============================] - 115s 917ms/step - loss: 0.3668 - accuracy: 0.8370 - val_loss: 0.3474 - val_accuracy: 0.8318
Epoch 3/15
126/126 [==============================] - ETA: 0s - loss: 0.3345 - accuracy: 0.8504
Epoch 3: val_loss improved from 0.34736 to 0.31211, saving model to /content/drive/MyDrive/IOH/spam_or_not/checkpoint/model.03-0.3121.h5
126/126 [==========================

In [73]:
teks_spam = "DISNAKERJA and others share their thoughts on LinkedIn"
model.predict([teks_spam])

1/1 [==============================] - 1s 791ms/step


array([[0.53002256]], dtype=float32)

In [80]:
teks = [
        "Bangkit Academy led by Google, Tokopedia, Gojek, & Traveloka and others share their thoughts on LinkedIn",
        "Kanao, welcome to Samsung",
        "[educba.com] Angular JS Certification Training Course (9 Courses, 5+ Projects)",
        "PTEB UTS 07111940000001",
        "Lomba Desain IG : itsmeeeaow",
        "Photo from Vanny Ezha",
        "SPCS1_tugas03_07111940000001",
        "Pendaftaran Akun Kampus Merdeka",
]
model.predict(teks)

1/1 [==============================] - 0s 69ms/step


array([[0.9920954 ],
       [0.12292992],
       [0.24726641],
       [0.28932017],
       [0.53702146],
       [0.34751543],
       [0.51677513],
       [0.09848826]], dtype=float32)

In [75]:
benar = []
for tes,lab in test.take(100):
  # print(tes.numpy().decode())
  if sum(model.predict([tes.numpy().decode()])) >= 0.5 and lab.numpy() == 1:
    benar.append(1)
  elif sum(model.predict([tes.numpy().decode()])) <= 0.5 and lab.numpy() == 1:
    pass
  elif sum(model.predict([tes.numpy().decode()])) >= 0.5 and lab.numpy() == 0:
    # print(False)
    pass
  elif sum(model.predict([tes.numpy().decode()])) <= 0.5 and lab.numpy() == 0:
    # print(True)
    benar.append(1)

sum(benar)
  # print(model.predict([tes.numpy().decode()]))
  # print(lab.numpy())

1/1 [==============================] - 0s 38ms/step


90

In [76]:
model.save("/content/drive/MyDrive/IOH/spam_or_not/saved_model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/IOH/spam_or_not/saved_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/IOH/spam_or_not/saved_model/assets


In [79]:
import pickle

with open("/content/drive/MyDrive/IOH/spam_or_not/pickle/model_spam.pkl","wb") as f:
  pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://ad024f4a-4277-4b0c-a278-6688a21e7241/assets


INFO:tensorflow:Assets written to: ram://ad024f4a-4277-4b0c-a278-6688a21e7241/assets


## Keterangan : lebih dari 0.5 berpotensi spam, kurang dari tersebut tidak berpotensi spam

In [91]:
model = tf.keras.models.load_model("/content/drive/MyDrive/IOH/spam_or_not/saved_model/")

In [97]:
teks = [
        "Bangkit Academy led by Google, Tokopedia, Gojek, & Traveloka and others share their thoughts on LinkedIn",
        "Kanao, welcome to Samsung",
        "[educba.com] Angular JS Certification Training Course (9 Courses, 5+ Projects)",
        "PTEB UTS 07111940000001",
        "Lomba Desain IG : itsmeeeaow",
        "Photo from Vanny Ezha",
        "SPCS1_tugas03_07111940000001",
        "Pendaftaran Akun Kampus Merdeka",
]

model(tf.constant(teks))

<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[0.9920954 ],
       [0.12292992],
       [0.24726641],
       [0.28932017],
       [0.53702146],
       [0.34751543],
       [0.51677513],
       [0.09848826]], dtype=float32)>